In [1]:
# !pip3 install pdfminer.six

In [2]:
# pip3 uninstall pdfminer
# !pip uninstall pdfminer.six
# pip3 install pdfminer

In [17]:
import io
from io import BytesIO

from pdfminer.converter import TextConverter, HTMLConverter
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

pdf_path = "example.pdf"

input_file = pdf_path

i_f = open(input_file,'rb')
resMgr = PDFResourceManager()
retData = io.BytesIO()
codec = 'utf-8'
TxtConverter = HTMLConverter(resMgr,retData, laparams= LAParams(), codec=codec)

interpreter = PDFPageInterpreter(resMgr,TxtConverter)
for page in PDFPage.get_pages(i_f):
    interpreter.process_page(page)

html_tags = retData.getvalue().decode("utf-8")
print(html_tags.replace('-\n<br>',''))


<html><head>
<meta http-equiv="Content-Type" content="text/html; charset=utf-8">
</head><body>
<span style="position:absolute; border: gray 1px solid; left:0px; top:50px; width:595px; height:842px;"></span>
<div style="position:absolute; top:50px;"><a name="1">Page 1</a></div>
<div style="position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:42px; top:135px; width:154px; height:8px;"><span style="font-family: Helvetica; font-size:8px">O R I G I N S   O F   R E N A L   D I S E A S E S
<br></span></div><div style="position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:42px; top:158px; width:175px; height:13px;"><span style="font-family: Helvetica; font-size:13px">History of Diabetes Insipidus
<br></span></div><div style="position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:99px; top:187px; width:109px; height:7px;"><span style="font-family: Helvetica,Bold; font-size:7px">Giovanna Valenti</span><span style="font-family: Helvetica; font-si

In [18]:
import re
import pandas as pd
from bs4 import BeautifulSoup

# open the html file
# html = open(pathHTMLoutput, 'r')


soup = BeautifulSoup(html_tags)

font_spans = [data for data in soup.select('span') if 'font-size' in str(data)]
output = []
for span in font_spans:
    fonts_size = re.search(r'(?is)(font-size:)(.*?)(px)', str(span.get('style'))).group(2)
    fonts_family = re.search(r'(?is)(font-family:)(.*?)(;)', str(span.get('style'))).group(2)

    # split fonts_family into fonts-type and fonts-style
    try:
        fonts_type = fonts_family.strip().split(',')[0]
        fonts_style = fonts_family.strip().split(',')[1]
    except IndexError:
        fonts_type = fonts_family.strip()
        fonts_style = None

    output.append((str(span.text).strip(), fonts_size.strip(), fonts_type, fonts_style))

# create dataframe
df = pd.DataFrame(output, columns = ['text', 'fonts_size', 'fonts_type', 'fonts_style'])
df["fonts_size"] = pd.to_numeric(df["fonts_size"])


In [29]:
df

,text,fonts_size,fonts_type,fonts_style,num_words_text
0,O R I G I N S O F R E N A L D I S E A S E S,8,Helvetica,None,22
1,History of Diabetes Insipidus,13,Helvetica,None,4
2,Giovanna Valenti,7,Helvetica,Bold,2
3,", Grazia Tamma",7,Helvetica,None,3
4,"Department of Biosciences, Biotechnologies and...",7,Helvetica,None,12
...,...,...,...,...,...
227,"[42] Mulders SM, Bichet DG, Rijss JP et al. An...",7,Helvetica,None,22
228,"[43] Li W, Zhang Y, Bouley R et al. Simvastati...",7,Helvetica,None,36
229,"[44] Procino G, Barbieri C, Carmosino M et al....",7,Helvetica,None,38
230,"[45] Los EL, Deen PM, Robben JH et al. Potenti...",7,Helvetica,None,32


In [30]:
df.dtypes

text              object
fonts_size         int64
fonts_type        object
fonts_style       object
num_words_text     int64
dtype: object

In [62]:
search_text = "References"
df[df['text'] == search_text]

,text,fonts_size,fonts_type,fonts_style,num_words_text
178,References,8,Helvetica,None,1


In [32]:
#BOLD
search_text = "Bold"
df[df['fonts_style'] == search_text]

,text,fonts_size,fonts_type,fonts_style,num_words_text
2,Giovanna Valenti,7,Helvetica,Bold,2
10,Key words:,7,Helvetica,Bold,2
12,",",7,Helvetica,Bold,1
14,",",7,Helvetica,Bold,1
16,",",7,Helvetica,Bold,1
18,",",7,Helvetica,Bold,1


In [63]:
df.iloc[179]

text              [1] Babey M, Kopp P, Robertson GL et al. Famil...
fonts_size                                                        7
fonts_type                                                Helvetica
fonts_style                                                    None
num_words_text                                                   24
Name: 179, dtype: object

In [34]:
df["num_words_text"] = df["text"].apply(lambda n: len(n.split()))

In [35]:
df

,text,fonts_size,fonts_type,fonts_style,num_words_text
0,O R I G I N S O F R E N A L D I S E A S E S,8,Helvetica,None,22
1,History of Diabetes Insipidus,13,Helvetica,None,4
2,Giovanna Valenti,7,Helvetica,Bold,2
3,", Grazia Tamma",7,Helvetica,None,3
4,"Department of Biosciences, Biotechnologies and...",7,Helvetica,None,12
...,...,...,...,...,...
227,"[42] Mulders SM, Bichet DG, Rijss JP et al. An...",7,Helvetica,None,22
228,"[43] Li W, Zhang Y, Bouley R et al. Simvastati...",7,Helvetica,None,36
229,"[44] Procino G, Barbieri C, Carmosino M et al....",7,Helvetica,None,38
230,"[45] Los EL, Deen PM, Robben JH et al. Potenti...",7,Helvetica,None,32


In [61]:
df[df['fonts_size'] == 10]

,text,fonts_size,fonts_type,fonts_style,num_words_text
20,Introduction,10,Helvetica,None,1
41,History of Diabetes Insipidus: a disease\niden...,10,Helvetica,None,11
67,Central Diabetes Insipidus is distinct from\nN...,10,Helvetica,None,9
81,Purification and synthesis of vasopressin,10,Helvetica,None,5
95,Cloning of the V2R and identification of\npati...,10,Helvetica,None,11
123,Cloning of the AQP2 water channels and\nidenti...,10,Helvetica,None,13
167,Summary and Conclusions,10,Helvetica,None,3


In [56]:
df_count = df[['fonts_size','num_words_text']]

df_count.groupby(['fonts_size']).sum()

,num_words_text
fonts_size,
5,1
6,182
7,1276
8,286
9,2069
10,53
13,4


In [65]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.layout import LAParams
from pdfminer.converter import TextConverter
from pdfminer.pdfpage import PDFPage
import io
import urllib.request
import requests


def pdf_to_text(pdf_file):
    text_memory_file = io.StringIO()

    rsrcmgr = PDFResourceManager()
    device = TextConverter(rsrcmgr, text_memory_file, laparams=LAParams())
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    # get first 3 pages of the pdf file
    for page in PDFPage.get_pages(pdf_file, pagenos=(0, 1, 2)):
        interpreter.process_page(page)
    text = text_memory_file.getvalue()
    text_memory_file.close()
    return text

# # online pdf to text by urllib
# online_pdf_file=urllib.request.urlopen('http://www.dabeaz.com/python/UnderstandingGIL.pdf')
# pdf_memory_file=io.BytesIO()
# pdf_memory_file.write(online_pdf_file.read())
# print(pdf_to_text(pdf_memory_file))


# online pdf to text by requests
response = requests.get('http://www.dabeaz.com/python/UnderstandingGIL.pdf')
pdf_memory_file = io.BytesIO()
pdf_memory_file.write(response.content)
print(pdf_to_text(pdf_memory_file))

# extract metadata
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdftypes import resolve1

parser = PDFParser(pdf_memory_file)
doc = PDFDocument(parser)
metadata=doc.info[0]
for k in metadata:
    print(k, resolve1(metadata[k]))

Understanding the 
Python GIL
David Beazley
http://www.dabeaz.com

Presented at PyCon 2010
Atlanta, Georgia

Copyright (C) 2010,  David Beazley, http://www.dabeaz.com

1

Introduction
• As a few of you might know, C Python has a 

Global Interpreter Lock (GIL)

>>> import that
The Unwritten Rules of Python

1. You do not talk about the GIL.
2. You do NOT talk about the GIL.
3. Don't even mention the GIL. No seriously.
...

• It limits thread performance
• Thus, a source of occasional "contention"

Copyright (C) 2010,  David Beazley, http://www.dabeaz.com

2

An Experiment
• Consider this trivial CPU-bound function

def countdown(n):
    while n > 0:
        n -= 1

• Run it once with a lot of work

COUNT = 100000000   # 100 million
countdown(COUNT)

• Now, subdivide the work across two threads

t1 = Thread(target=countdown,args=(COUNT//2,))
t2 = Thread(target=countdown,args=(COUNT//2,))
t1.start(); t2.start()
t1.join(); t2.join()

Copyright (C) 2010,  David Beazley, http://www.dabeaz

In [71]:
import io
from io import BytesIO

from pdfminer.converter import TextConverter, HTMLConverter
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

input_file = "example.pdf"

i_f = open(input_file,'rb')
resMgr = PDFResourceManager()
retData = io.BytesIO()
codec = 'utf-8'
TxtConverter = HTMLConverter(resMgr,retData, laparams= LAParams(), codec=codec)

interpreter = PDFPageInterpreter(resMgr,TxtConverter)
for page in PDFPage.get_pages(i_f):
    interpreter.process_page(page)

html_tags = retData.getvalue().decode("utf-8")
# print(html_tags.replace('-\n<br>',''))


In [72]:
# extract metadata
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdftypes import resolve1

parser = PDFParser(html_tags)
doc = PDFDocument(parser)
metadata=doc.info[0]
for k in metadata:
    print(k, resolve1(metadata[k]))

AttributeError: 'str' object has no attribute 'seek'